In [4]:
import pandas as pd
import numpy as np

from influxdb_client import InfluxDBClient, Point, Dialect

import re
import time
import datetime

import warnings
from influxdb_client.client.warnings import MissingPivotFunction

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.colors as colors

import pandasql as ps
import sqlite3

import csv

import warnings 
from influxdb_client.client.warnings import MissingPivotFunction
warnings.simplefilter("ignore", MissingPivotFunction)

import gc

pd.set_option('display.max_rows', 500)

In [6]:
data = pd.read_csv('./overhead.csv', header=0,  index_col=0)
final = pd.DataFrame(data)

final.head(10)

,experiment,start,end,topology,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,bandwidth,messageOverhead
0,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518,unl,8,6,12,4,8,2,0.25,1.0,285.933511,2473589
1,391,2023-10-04 19:25:37.1533399,2023-10-04 19:55:37.1555856,unl,8,6,12,4,8,2,0.25,1.0,195.675311,1681977
2,392,2023-10-04 19:59:12.6589825,2023-10-04 20:29:12.6608307,unl,8,6,12,4,8,2,0.25,1.0,86.268800,1379082
3,393,2023-10-04 20:32:47.3098616,2023-10-04 21:02:47.3149178,unl,8,6,12,4,8,2,0.25,3.0,108.486362,1948016
4,394,2023-10-04 21:06:22.7755634,2023-10-04 21:36:22.7788800,unl,8,6,12,4,8,2,0.25,3.0,94.085382,1843504
5,395,2023-10-04 21:39:58.6528165,2023-10-04 22:09:58.6545536,unl,8,6,12,4,8,2,0.25,3.0,36.928853,687571
6,396,2023-10-04 22:13:33.8146704,2023-10-04 22:43:33.8174094,unl,16,8,20,16,8,4,0.50,1.0,79.777656,1428674
7,397,2023-10-04 22:47:09.3837110,2023-10-04 23:17:09.3868442,unl,16,8,20,16,8,4,0.50,1.0,69.534879,1428254
8,398,2023-10-04 23:20:44.6001719,2023-10-04 23:50:44.6033804,unl,16,8,20,16,8,4,0.50,1.0,7.107765,141417
9,402,2023-10-05 01:35:05.2471450,2023-10-05 02:05:05.2498822,unl,16,8,20,16,8,4,0.50,0.5,75.661552,289177


In [8]:
publishMessage = pd.DataFrame()
deliverMessage = pd.DataFrame()

for index, row in final.iterrows():
    # print(row["start"], row["end"], row["experiment"])
    execs = exps.loc[(exps["topology"] == row["topology"]) & (exps["d"] == row["d"]) &
                            (exps["dlo"] == row["dlo"]) & (exps["dhi"] == row["dhi"]) &
                            (exps["dscore"] == row["dscore"]) & (exps["dlazy"] == row["dlazy"]) &
                            (exps["dout"] == row["dout"]) & (exps["gossipFactor"] == row["gossipFactor"]) &                    
                            (exps["interval"] == row["interval"])]

    for idx, exec in execs.iterrows():
        # print(exec["startUnix"], exec["endUnix"])
        query_mess = from_influx(url, token, org, "deliverMessage", exec["startUnix"], exec["endUnix"],"messageID")
        query_pub = from_influx(url, token, org, "publishMessage", exec["startUnix"], exec["endUnix"],"messageID")
    
        query_mess["identifier"] = row["identifier"]
        query_pub["identifier"] = row["identifier"]

        query_mess["experiment"] = exec["experiment"]
        query_pub["experiment"] = exec["experiment"]
    
        query_mess["_time"] = pd.to_datetime(query_mess["_time"])
        query_pub["_time"] = pd.to_datetime(query_pub["_time"])
    
        deliverMessage = pd.concat([query_mess, deliverMessage])
        publishMessage = pd.concat([publishMessage, query_pub])

        del query_pub
        del query_mess

        gc.collect()

deliverMessage.head(10)


,_time,messageID,identifier,experiment
0,2023-08-30 02:46:59.183105+00:00,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...,144,295
1,2023-08-30 02:46:59.183139+00:00,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...,144,295
2,2023-08-30 02:46:59.183960+00:00,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...,144,295
3,2023-08-30 02:46:59.184174+00:00,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...,144,295
4,2023-08-30 02:46:59.184592+00:00,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...,144,295
5,2023-08-30 02:46:59.184598+00:00,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...,144,295
6,2023-08-30 02:46:59.184767+00:00,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...,144,295
7,2023-08-30 02:46:59.184975+00:00,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...,144,295
8,2023-08-30 02:46:59.185165+00:00,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...,144,295
9,2023-08-30 02:46:59.185341+00:00,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...,144,295


In [11]:
data = pd.read_csv('../datasets/timeSeries1s.csv', header=0,  index_col=0)
message = pd.DataFrame(data)

# idx = [0.0, 3.0]
# message = message[message["type"].isin(idx)]
# message = message.drop(columns=["identifier"])

message.loc[message["type"] == 0].tail(10)

,identifier,experiment,type,_time,count


In [9]:
publishMessage.head(10)


,_time,messageID,identifier,experiment
0,2023-08-25 17:04:39.544928+00:00,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...,2,152
1,2023-08-25 17:04:39.545138+00:00,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...,2,152
2,2023-08-25 17:04:39.545336+00:00,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...,2,152
3,2023-08-25 17:04:39.545570+00:00,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...,2,152
4,2023-08-25 17:04:39.545820+00:00,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...,2,152
5,2023-08-25 17:04:39.546015+00:00,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...,2,152
6,2023-08-25 17:04:39.546236+00:00,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...,2,152
7,2023-08-25 17:04:39.546421+00:00,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...,2,152
8,2023-08-25 17:04:40.063126+00:00,ACQIARIgZTwM9zqvuGbntFGs3WCNz/6u9RqIjPBSg9JdSf...,2,152
9,2023-08-25 17:04:40.063506+00:00,ACQIARIgZTwM9zqvuGbntFGs3WCNz/6u9RqIjPBSg9JdSf...,2,152


In [12]:

# def calcAverageTime(publish, received, expTime, parameter):
expTime = exps
publish = publishMessage
received = deliverMessage

publish = publish[['_time', 'messageID', 'identifier','experiment']]
received = received[['_time', 'messageID', 'identifier','experiment']]

joined = publish.merge(received, on=['messageID', 'identifier', 'experiment'])
joined['diff'] = ((joined['_time_y'] - joined['_time_x'])/ pd.Timedelta(microseconds=1)).astype(int)
joined = joined.loc[joined["diff"] >= 0].dropna()
joined.head(10)

,_time_x,messageID,identifier,experiment,_time_y,diff
0,2023-08-25 17:04:39.544928+00:00,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...,2,152,2023-08-25 17:04:39.546712+00:00,1784
1,2023-08-25 17:04:39.544928+00:00,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...,2,152,2023-08-25 17:04:39.546809+00:00,1881
2,2023-08-25 17:04:39.544928+00:00,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...,2,152,2023-08-25 17:04:39.547608+00:00,2680
3,2023-08-25 17:04:39.544928+00:00,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...,2,152,2023-08-25 17:04:39.547969+00:00,3041
4,2023-08-25 17:04:39.544928+00:00,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...,2,152,2023-08-25 17:04:39.548085+00:00,3157
5,2023-08-25 17:04:39.544928+00:00,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...,2,152,2023-08-25 17:04:39.548558+00:00,3630
6,2023-08-25 17:04:39.544928+00:00,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...,2,152,2023-08-25 17:04:39.548572+00:00,3644
7,2023-08-25 17:04:39.544928+00:00,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...,2,152,2023-08-25 17:04:39.549385+00:00,4457
8,2023-08-25 17:04:39.544928+00:00,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...,2,152,2023-08-25 17:04:39.549517+00:00,4589
9,2023-08-25 17:04:39.544928+00:00,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...,2,152,2023-08-25 17:04:39.550178+00:00,5250


In [14]:
#Average propagation time per interval
df = joined.drop(columns=['_time_x', '_time_y', 'messageID'])

avgPropExp = df.groupby(['experiment', 'identifier']).agg('mean')
avgPropExp.reset_index(inplace=True)
avgPropExp = avgPropExp.drop(columns=['experiment'])
# avgPropExp.head(10)

avgProp = avgPropExp.groupby(['identifier']).agg({'diff':['mean','std']}).fillna(0)
avgProp.columns = avgProp.columns.droplevel(0)
avgProp.reset_index(inplace=True)

avgProp.head(10)

,identifier,mean,std
0,2,1451.646409,1050.668381
1,3,986.740298,20.491311
2,6,1325.467710,93.967445
3,9,1294.990615,294.123534
4,12,1315.527934,262.677875
5,15,1524.225677,51.515811
6,18,3237.055361,2525.238950
7,21,1788.039572,306.026796
8,24,1597.645548,326.575399
9,27,1157.719611,261.649231


In [16]:
finalProp = final.merge(avgProp, on='identifier')
finalProp = finalProp.rename(columns={'mean' : 'propagationTime', 'std' : 'stddev_propagationTime'})#.drop(columns=['std'])

finalProp.to_csv('propagation.csv')
finalProp.head(10)

,identifier,topology,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,n_nodes,bandwidth,propagationTime,stddev_propagationTime
0,2,unl,8,6,12,4,8,2,0.25,1.0,23,349.473952,1451.646409,1050.668381
1,3,unl,8,6,12,4,8,2,0.25,0.5,23,413.648073,986.740298,20.491311
2,6,unl,8,6,12,4,8,2,0.25,30.0,23,1056.993541,1325.467710,93.967445
3,9,unl,8,6,12,4,8,2,0.25,3.0,23,744.672331,1294.990615,294.123534
4,12,unl,24,6,12,4,8,2,0.25,1.0,23,824.463965,1315.527934,262.677875
5,15,unl,12,6,12,4,8,2,0.25,1.0,23,961.139534,1524.225677,51.515811
6,18,unl,6,6,12,4,8,2,0.25,1.0,23,621.270477,3237.055361,2525.238950
7,21,unl,8,2,12,4,8,2,0.25,1.0,23,604.560808,1788.039572,306.026796
8,24,unl,8,3,12,4,8,2,0.25,1.0,23,360.859540,1597.645548,326.575399
9,27,unl,8,8,12,4,8,2,0.25,1.0,23,287.544853,1157.719611,261.649231
